<a href="https://colab.research.google.com/github/ClaudeCoulombe/VIARENA/blob/master/Labos/Lab-Ecorces_Arbres/IdEcorces-ResConv-AnalyseDErreurs-Colab.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


### Rappel - Fonctionnement d'un carnet web iPython

* Pour exécuter le code contenu dans une cellule d'un carnet iPython, cliquez dans la cellule et faites (⇧↵, shift-enter) 
* Le code d'un carnet iPython s'exécute séquentiellement de haut en bas de la page. Souvent, l'importation d'une bibliothèque Python ou l'initialisation d'une variable est préalable à l'exécution d'une cellule située plus bas. Il est donc recommandé d'exécuter les cellules en séquence. Enfin, méfiez-vous quand vous faites des retours en arrière, cela peut réinitialiser certaines variables.

SVP, déployez toutes les cellules en sélectionnant l'item « Développer les rubriques » de l'onglet « Affichage ».

# Analyse d'erreur
## Identification d'arbres à partir de leur écorce
### Réseau convolutif, apprentissage par transfert et amplification des données

### Inspiration et droits d'auteur

Ce laboratoire s'inspire de plusieurs oeuvres en logiciels libres qui ont été transformées dont:

<a href="https://www.tensorflow.org/tutorials/images/transfer_learning" target='_blank'>Transfer learning and fine-tuning</a> - site Google / Tutoriels TensorFlow

<a href="https://www.tensorflow.org/tutorials/images/data_augmentation" target='_blank'>Data augmentation</a> - site Google / Tutoriels TensorFlow

##### Copyright (c) 2017, François Chollet  
##### Copyright (c) 2019-2022, The TensorFlow Authors.
##### Copyright (c) 2022, Claude Coulombe

Le contenu de cette page est sous licence <a href="https://creativecommons.org/licenses/by/4.0/deed.fr" target='_blank'>Creative Commons Attribution 4.0 (CC BY 4.0)</a>,<br/>et les exemples de code sont sous <a href="https://www.apache.org/licenses/LICENSE-2.0" target='_blank'>licence Apache 2.0</a>.

#### Données

Les données sur les écorces d'arbres proviennent de <a href="https://data.mendeley.com/research-data/?search=barknet">BarkNet</a>, une banque en données ouvertes sous licence MIT de 23 000 photos d'écorces d'arbres en haute résolution prises avec des téléphones intelligents par une équipe d'étudiants et de chercheurs du <a href="https://www.sbf.ulaval.ca/" target='_blank'>Département des sciences du bois et de la forêt de l'Université Laval</a> à Québec.</p>

## Fixer le hasard pour la reproductibilité

La mise au point de réseaux de neurones implique certains processus aléatoires. Afin de pouvoir reproduire et comparer vos résultats d'expérience, vous fixez temporairement l'état aléatoire grâce à un germe aléatoire unique.

Pendant la mise au point, vous fixez temporairement l'état aléatoire pour la reproductibilité mais vous répétez l'expérience avec différents germes ou états aléatoires et prenez la moyenne des résultats.
<br/>

**Note** : Pour un système en production, vous ravivez simplement l'état  purement aléatoire avec l'instruction `GERME_ALEATOIRE = None`

In [ ]:
import os

# Définir un germe aléatoire
GERME_ALEATOIRE = 42

# Définir un état aléatoire pour Python
os.environ['PYTHONHASHSEED'] = str(GERME_ALEATOIRE)

# Définir un état aléatoire pour Python random
import random
random.seed(GERME_ALEATOIRE)

# Définir un état aléatoire pour NumPy
import numpy as np
np.random.seed(GERME_ALEATOIRE)

# Définir un état aléatoire pour TensorFlow
import tensorflow as tf
tf.random.set_seed(GERME_ALEATOIRE)

# Note: Retrait du comportement déterministe
# à cause de keras.layers.RandomContrast(...)
# dont il n'existe pas de version déterministe
# os.environ['TF_DETERMINISTIC_OPS'] = '1'
# os.environ['TF_CUDNN_DETERMINISTIC'] = '1'

print("Germe aléatoire fixé")

## Acquisition des données

L'analyse d'erreur requiert l'examen des données d'entraînement, il faut donc les obtenir.

Les <a href="https://www.kaggle.com/claudecoulombe/barknet" target='_blank'>données de BarkNet</a>  peuvent être téléchargées à partir du site de Kaggle. Mais vous allez utiliser l'IPA (<i>API</i>) de Kaggle pour accélérer les transferts de données.

In [ ]:
dict_arbres = {
    'BOJ' : "Betula alleghaniensis - Bouleau jaune - Yellow birch",
    'BOP' : "Betula papyrifera - Bouleau à papier - White birch",
    'CHR' : "Quercus rubra - Chêne rouge - Northern red oak",
    'EPB' : "Picea glauca - Épinette blanche - White spruce",
    'EPN' : " Picea mariana - Épinette noire - Black spruce",
    'EPO' : "Picea abies - Épinette de Norvège - Norway spruce",
    'EPR' : "Picea rubens - Épinette rouge - Red spruce",
    'ERB' : "Acer platanoides - Érable de Norvège - Norway maple",
    'ERR' : "Acer rubrum - Érable rouge - Red maple",
    'ERS' : "Acer saccharum - Érable à sucre - Sugar maple",
    'FRA' : "Fraxinus americana - Frêne d'Amérique - White ash",
    'HEG' : "Fagus grandifolia - Hêtre à grandes feuilles - American beech",
    'MEL' : "Larix laricina - Mélèze - Tamarack",
    'ORA' : "Ulmus americana - Orme d'Amérique - American elm",
    'OSV' : "Ostrya virginiana - Ostryer de Virginie - American hophornbeam",
    'PEG' : "Populus grandidentata - Peuplier à grandes dents - Big-tooth aspen",
    'PET' : "Populus tremuloides - Peuplier faux tremble - Quaking aspen",
    'PIB' : "Pinus strobus - Pin blanc - Eastern white pine",
    'PID' : "Pinus rigida - Pin rigide - Pitch pine",
    'PIR' : "Pinus resinosa - Pin rouge - Red pine",
    'PRU' : "Tsuga canadensis - Pruche du Canada - Eastern Hemlock",
    'SAB' : "Abies balsamea - Sapin Baumier - Balsam fir",
    'THO' : "Thuja occidentalis - Thuya occidental - Northern white cedar",
}

print("Dictionnaire mémorisé")

### Création des répertoires de données

Nous allons créer un répertoire de base `donnees`, un répertoire `lab_ecorces` où les données seront réparties en données d'entraînement, de validation et de test pour chaque classe cible.

Enfin, un répertoire `modeles` pour mémoriser les modèles une fois entraînés.

In [ ]:
try:
    os.mkdir("/content/donnees/")
except OSError:
    pass
try:
    os.mkdir("/content/lab_ecorces/")
except OSError:
    pass
try:
    os.mkdir("/content/modeles/")
except OSError:
    pass

### Utilisation de l'IPA (<i>API</i>) de Kaggle pour l'importation directe du jeu de données BarkNet

1. Commencez par installer la bibliothèque Python `kaggle`

In [ ]:
!pip3 install kaggle

2. Si ce n'est déjà fait, devenez membre de Kaggle avec votre adresse de courriel GMail:<br/>

<img src="https://cours.edulib.org/asset-v1:Cegep-Matane+VAERN.1FR+P2021+type@asset+block@Kaggle_API-1.png"/>

3. Maintenant, vous devez télécharger votre clé privée pour utiliser l'IPA de Kaggle.

4. Cliquez sur l'onglet « account » de votre profil Kaggle

<img src="https://cours.edulib.org/asset-v1:Cegep-Matane+VAERN.1FR+P2021+type@asset+block@Kaggle_API-2.png"/>

5. Sur la page « Account » cliquez sur le bouton « Create New API Token ».
    
<img style="margin-left:40px;" src="https://cours.edulib.org/asset-v1:Cegep-Matane+VAERN.1FR+P2021+type@asset+block@Kaggle_API-3.png"/>

6. Téléchargez votre clé privée « kaggle.json » pour l'IPA Kaggle dans un endroit temporaire sur votre poste de travail.

<img style="margin-left:40px;" src="https://cours.edulib.org/asset-v1:Cegep-Matane+VAERN.1FR+P2021+type@asset+block@Kaggle_API-5.png"/>

7. Maintenant, transférez (téléversez) votre clé privée « kaggle.json » dans votre environnement Colab.

La fenêtre de l'outil de fichiers de votre ordinateur s'ouvre alors. Allez chercher votre clé privée « kaggle.json » que vous avez sauvegardée sur votre  ordinateur.

<img style="margin-left:40px;" src="https://cours.edulib.org/asset-v1:Cegep-Matane+VAERN.1FR+P2021+type@asset+block@Colab_Importer_Fichier.png"/>


8. Créer à la racine un répertoire .kaggle et déplacez votre clé privée « kaggle.json » dans ce répertoire.

In [ ]:
!mkdir ~/.kaggle
!cp /content/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!ls ~/.kaggle -all

9. Maintenant téléchargez le jeu de données réduit « barknet-mini » de 1.6 Go ou le jeu de données « barknet » complet de 32 Go avec la commande suivante:

In [ ]:
# Attention! Jeu réduit de données 1.5 Go - plus rapide à télécharger et à traiter
!kaggle datasets download claudecoulombe/barknet-mini --unzip -p /content/donnees/
repertoire_entree = "/content/donnees/BarkNet-mini"

In [ ]:
# Attention! Jeu complet de données 32 Go - plus long à télécharger et à traiter
# retirer les commentaires ci-dessous pour activer le code  
# !kaggle datasets download claudecoulombe/barknet --unzip -p /content/donnees/
# repertoire_entree = "/content/donnees/BarkNet"

### Répartition des données

In [ ]:
# Installation des bibliothèques Python `split-folders` et `tqdm`
!pip3 install split-folders tqdm

In [ ]:
# Répartition des données d'entraînement, de validation et de tests
import splitfolders
import pathlib

#### répertoire des données une fois réparties
repertoire_donnees_reparties = "/content/lab_ecorces"
# => train, val, test

nombre_images = len(list(pathlib.Path(repertoire_entree).glob('*/*.jpg')))
print("Nombre total d'images:",nombre_images)

splitfolders.ratio(repertoire_entree, 
                   output=repertoire_donnees_reparties, 
                   seed=42, 
                   ratio = (0.65, 0.15, 0.20)
                   )

print("\nRépartition des données terminée!")


## Acquisition du modèle entraîné dont on veut analyser les erreurs

### Téléversement et décompression d'un modèle Keras sauvegardé sur un poste local

1. & 2. Vous allez téléverser le modèle d'identification d'écorces entraîné précédemment.

<img src="https://cours.edulib.org/asset-v1:Cegep-Matane+VAERN.1FR+P2021+type@asset+block@Colab_Importer_Fichier.png"/>

3. La fenêtre de l'outil de fichiers de votre ordinateur s'ouvre alors. Allez chercher le modèle d'identification d'écorces modele_....zip que vous avez sauvegardé précédemment sur votre ordinateur local.

Attention! Le téléchargement peut prendre plusieurs minutes. Assurez-vous que le fichier est entièrement téléversé et que l'icône d'état du téléversement (un cercle jaune, comme ci-dessous) disparaisse. 

<img style="margin-left:0px" src="https://cours.edulib.org/asset-v1:Cegep-Matane+VAERN.1FR+P2021+type@asset+block@Colab_Importer_Fichier-3.png"/>

4. Décompressez le fichier modele_....zip en exécutant la commande ci-dessous:<br/>


In [ ]:
!unzip /content/modele_*.zip -d /content/modeles && rm /content/modele_*.zip

## Chargement du modèle entraîné dont on veut analyser les erreurs

In [ ]:
import os
import numpy as np
import keras
print("Version de Keras:",keras.__version__)
import tensorflow as tf
print("Version de TensorFlow :",tf.__version__)


In [ ]:
chemin_modele_sauvegarde = "/content/modeles/"

modele_de_transfert = tf.keras.models.load_model(chemin_modele_sauvegarde)

In [ ]:
print("Architecture du modèle préentraîné")
modele_de_transfert.summary()

## Prédiction sur les données d'entraînement

Pour procéder à l'analyse des erreurs, vous allez débuter par l'examen des erreurs sur les données d'entraînement. 

Par la suite, vous utiliserez des données fraîches qui n'ont pas servi à entraîner le modèle. Ces données devraient être disctinctes des données de test qui doivent être conservées pour l'évaluation finale du modèle.

### Prétraitement des données

In [ ]:
REPERTOIRE_ENTRAINEMENT = "/content/lab_ecorces/train/"
TAILLE_LOT = 32
HAUTEUR_IMAGE = 150
LARGEUR_IMAGE = 150
TAILLE_IMAGE = (HAUTEUR_IMAGE, LARGEUR_IMAGE)
NOMBRE_CANAUX = 3

couches_amplification = tf.keras.Sequential([
    # Retournement horizontal - gauche / droite
    keras.layers.RandomFlip("horizontal"),
    # Retournement vertical - haut / bas
    keras.layers.RandomFlip("vertical"),
    # Rotation
    keras.layers.RandomRotation(0.1),
    # Agrandissement / zoom
    keras.layers.RandomZoom(0.3),
    # Variation du contraste de l'image
    keras.layers.RandomContrast(0.3),
])

couches_normalisation = keras.Sequential([
    # Redimensionnement de l'image
    keras.layers.Resizing(HAUTEUR_IMAGE,LARGEUR_IMAGE),
    # Changement d'échelle de luminosité
    keras.layers.Rescaling(1./255)
])

AUTOTUNE = tf.data.AUTOTUNE

def pretraitement(jeu_donnees, melanger=False, normaliser=False, amplifier=False):
    if melanger:
        jeu_donnees = jeu_donnees.shuffle(1000)
                
    # Normaliser les jeux de données
    if normaliser:
        jeu_donnees = jeu_donnees.map(lambda x, y: (couches_normalisation(x), y),
                                    num_parallel_calls=AUTOTUNE
                                    )
    # Amplifier seulement les données d'entraînement
    if amplifier:
        jeu_donnees = jeu_donnees.map(lambda x, y: (couches_amplification(x,training=True), y),
                                      num_parallel_calls=AUTOTUNE
                                     )
    # Utiliser des tampons de préextraction sur tous les jeux de données
    return jeu_donnees.prefetch(buffer_size=AUTOTUNE)

print("Fonction de prétraitement prête!")

In [ ]:
donnees_entrainement = tf.keras.utils.image_dataset_from_directory(REPERTOIRE_ENTRAINEMENT,
                                                                   batch_size=TAILLE_LOT,
                                                                   image_size=TAILLE_IMAGE,
                                                                   shuffle=False)

donnees_entrainement_normalisees = pretraitement(donnees_entrainement,
                                                 melanger=False,
                                                 normaliser=True,
                                                 amplifier=False)

### Inférence sur les donnés d'entraînement*

In [ ]:
les_images = [] 
etiquettes_vraies = []  
etiquettes_predites = [] 
liste_noms_classes = donnees_entrainement.class_names
print("Noms de classes:\n",liste_noms_classes)
print()
# boucler sur le jeu de données d'entraînement
for lot_images, lot_etiquettes in donnees_entrainement_normalisees: 
   # accumuler les images
   les_images.append(lot_images)
   # accumuler les vraies étiquettes
   etiquettes_vraies.append(lot_etiquettes)
   # faire des prédictions
   predictions = modele_de_transfert.predict(lot_images)
   # accumuler les étiquettes prédites
   etiquettes_predites.append(np.argmax(predictions, axis = - 1))
# convertir les listes d'étiquettes en tenseurs
liste_vraies_etiquettes_entrainement = tf.concat([item for item in etiquettes_vraies], axis = 0)
liste_etiquettes_predites_entrainement = tf.concat([item for item in etiquettes_predites], axis = 0)
liste_images = tf.concat([item for item in les_images], axis = 0)
print("Étiquettes prêtes!")


### Mesure d'exactitude

In [ ]:
from sklearn import metrics

exactitude_test = metrics.accuracy_score(liste_vraies_etiquettes_entrainement, liste_etiquettes_predites_entrainement)
print("Exactitude:   %0.2f" % exactitude_test)


### Matrice de confusion

In [ ]:
# https://stackoverflow.com/questions/65618137/confusion-matrix-for-multiple-classes-in-python

import matplotlib.pyplot as plt
import itertools
%matplotlib inline

def afficher_matrice_de_confusion(matrice_confusion_brute, classes,
                          normalisation=False,
                          titre='Matrice de confusion',
                          carte_des_couleurs=plt.cm.Blues):
    plt.figure(figsize=(14,12))
    plt.imshow(matrice_confusion_brute, interpolation='nearest', cmap=carte_des_couleurs)
    plt.title(titre)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalisation:
        matrice_confusion_brute = matrice_confusion_brute.astype('float') / matrice_confusion_brute.sum(axis=1)[:, np.newaxis]
        print("Matrice de confusion normalisée")
    else:
        print('Matrice de confusion non normalisée')

    seuil = matrice_confusion_brute.max() / 2.
    for i, j in itertools.product(range(matrice_confusion_brute.shape[0]), range(matrice_confusion_brute.shape[1])):
        plt.text(j, i, matrice_confusion_brute[i, j],
                 horizontalalignment="center",
                 color="white" if matrice_confusion_brute[i, j] > seuil else "black")

    plt.tight_layout()
    plt.ylabel('Vraies étiquettes')
    plt.xlabel('Étiquettes prédites')

print("Code affichage matrice de confusion")


In [ ]:
matrice_confusion_brute = metrics.confusion_matrix(liste_vraies_etiquettes_entrainement, liste_etiquettes_predites_entrainement)
afficher_matrice_de_confusion(matrice_confusion_brute, classes=liste_noms_classes)


### Rapport de classification

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(liste_vraies_etiquettes_entrainement, 
                            liste_etiquettes_predites_entrainement, 
                            target_names=liste_noms_classes))


## Examen manuel et visualisation des erreurs

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

def trouver_nom_arbre(index):
    id_arbre = liste_noms_classes[int(index)]
    return dict_arbres[id_arbre].split("-")[1]

def trouver_id_arbre(index):
    return liste_noms_classes[int(index)]

nombre_erreurs = 0
images_mal_classees = []
fig = plt.figure(figsize=(12,4))
for (index,etiq_vraie,etiq_pred) in zip(range(len(liste_images)),
                                              liste_vraies_etiquettes_entrainement,
                                              liste_etiquettes_predites_entrainement):
    etiq_pred = etiq_pred.numpy()
    etiq_vraie = etiq_vraie.numpy()
    if (etiq_vraie != etiq_pred):
        print("_"*80)
        nombre_erreurs += 1
        print("*** ERREUR*** Prédiction:",etiq_pred,"-",trouver_id_arbre(etiq_pred),"-",trouver_nom_arbre(str(etiq_pred)),
              "- Vraie:",etiq_vraie,"-",trouver_id_arbre(etiq_vraie),"-",trouver_nom_arbre(str(etiq_vraie)))
        chemin_image_originale = donnees_entrainement.file_paths[index]
        print("Chemin image originale:",chemin_image_originale)
        images_mal_classees.append(chemin_image_originale)
        image_originale = mpimg.imread(chemin_image_originale)
        plt.axis('Off')
        plt.imshow(image_originale)
        plt.show()
print("_"*80)
print("Nombre total d'erreurs:",nombre_erreurs)


## Résultats 

Typiquement, l'analyse d'erreur permet de découvrir des anomalies, des données aberrantes ou mal étiquetées que vous allez corriger, remplacer ou simplement écarter. L'analyse d'erreur peut également déboucher sur de nouvelles étapes de prétraitement de vos données.

Ici, nous n'avons pu procéder à cette analyse car nous n'avions pas d'expertise en identification d'écorces. 

Nous avons toutefois noté que des espèces d'arbres manquaient probablement de variété dans leurs données. En fait, il n'y a pas suffisamment de spécimens différents. Il n'y a qu'un seul spécimen d'érable de Norvège (Acer platanoides), 3 spécimens de Peuplier à grandes dents (Populus grandidentata) et 4 spécimens de Pin rigide (Pinus rigida). La solution simple est de supprimer ces trois classs, ce qui augmente l'exactitude d'environ 3%. Pour bien faire, il faudrait ajouter quelques spécimens en allant sur le terrain ou en trouvant de bonnes photos sur la Toile.

De plus, il nous a semblé que certaines photos floues ou mal éclairées pouvaient être à l'origine de certaines erreurs de classification. Cela suggère un prétraitement photographique de ces images.

In [ ]:
print("Fin de l'exécution du carnet IPython")